In [ ]:
import base64
import openai
from openai import OpenAI
import json
from tqdm.auto import tqdm
import os
from collections import defaultdict

def sample_indices(n, total_frames):
    return [int(round(i * (total_frames - 1) / (n - 1) + 1)) for i in range(n)]

batch = []

prompts = [2, 4, 5, 6, 8, 11, 12, 13, 15, 16, 18]

filtered_119 = "/n/fs/visualai-scr/temp_LLP/ellie/slowfast_kinetics/filtered_119.json"

with open(filtered_119, "r") as f:
    filtered_119_prompts = json.load(f)


video_dict = defaultdict(list)


for prompt_num in prompts:

    json_path = f"/n/fs/visualai-scr/temp_LLP/ellie/slowfast_kinetics/gpt/4o-mini_automated_prompt_{prompt_num}_75_mini_actionswap/filtered_prompts.json"

    with open(json_path, "r") as f:
        data = json.load(f)

    for item in tqdm(data):
        prompt = item["prompt"]
        action_swap_path = item["action_swap_path"]

        if action_swap_path in filtered_119_prompts: #skip the 119 filtered prompts
            # print("skipping")
            continue

        lst = [prompt_num, prompt]
        video_dict[action_swap_path].append(tuple(lst))

num_prompts = []
for key, item in video_dict.items():
    num_prompts.append(len(item))

print(sum(num_prompts))

#Go through each video data
# Figure out which prompts we need 

for key, item in video_dict[action_swap_path]:
    action_swap_path = key

    




100%|██████████| 198/198 [00:00<00:00, 633948.24it/s]


100%|██████████| 1864/1864 [00:00<00:00, 535932.46it/s]

17433


In [3]:
import base64
import openai
from openai import OpenAI
import json
from tqdm.auto import tqdm
import os
import base64
import openai
from openai import OpenAI
import json
from tqdm.auto import tqdm
import os
from collections import defaultdict

def sample_indices(n, total_frames):
    return [int(round(i * (total_frames - 1) / (n - 1) + 1)) for i in range(n)]

batch = []
batch_length = 0

prompts = [2, 4, 5, 6, 8, 11, 12, 13, 15, 16, 18]

filtered_119 = "/n/fs/visualai-scr/temp_LLP/ellie/slowfast_kinetics/filtered_119.json"

with open(filtered_119, "r") as f:
    filtered_119_prompts = json.load(f)


video_dict = defaultdict(list)

count = 1

for prompt_num in prompts:

    json_path = f"/n/fs/visualai-scr/temp_LLP/ellie/slowfast_kinetics/gpt/4o-mini_automated_prompt_{prompt_num}_75_mini_actionswap/filtered_prompts.json"

    with open(json_path, "r") as f:
        data = json.load(f)

    for item in tqdm(data):
        prompt = item["prompt"]
        action_swap_path = item["action_swap_path"]

        if action_swap_path in filtered_119_prompts: #skip the 119 filtered prompts
            # print("skipping")
            continue

        video_dict[action_swap_path].append([prompt_num, prompt])



num_prompts = []
for key, values in tqdm(video_dict.items()):
    # print(key, values)
    # num_prompts.append(len(item.values()))

    action_swap_path = key


    total_frames = sum(1 for f in os.listdir(action_swap_path) if os.path.isfile(os.path.join(action_swap_path, f))) - 2
    # print("total frames", total_frames)
    indices = sample_indices(8, total_frames)
    image_paths = [os.path.join(action_swap_path, f"{index:06d}.jpg") for index in indices]

    loaded_images = [
                        {
                            "type": "image_url",
                            'image_url':  {
                                "url": f"data:image/jpeg;base64,{base64.b64encode(open(imgurl, 'rb').read()).decode('ascii')}",
                                "detail":"low"
                            }

                        } for imgurl in image_paths
                    ]


    #Go through each prompt

    for elem in values: #Gp through all the prompts for this video
        message = [
                {
                    "role": "system",
                    "content": "You are a helpful system"
                },
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": elem[1]
                        },
                    ] + loaded_images
                }
            ]


        json_file = json.dumps(
            {
                'custom_id': f"{elem[0]}:{action_swap_path}",
                'method': 'POST',
                'url': '/v1/chat/completions',
                'body': {
                    'model': 'gpt-4o-mini',
                    'messages': message,
                }
            }
        )
        length = len(json_file) / 1000 / 1000

        if length + batch_length > 190:
            file_name = f"batches/batch_{count}.jsonl"
            os.makedirs("batches", exist_ok=True)
            with open(file_name, 'w') as f:
                for item in batch:
                    f.write(item + '\n')

            batch = []
            count += 1
            batch.append(
                json_file
            )
            batch_length = length
        else:
            batch.append(
                json_file
            )
            batch_length += length


if len(batch) > 0:
    file_name = f"batches/batch_{count}.jsonl"
    os.makedirs("batches", exist_ok=True)
    with open(file_name, 'w') as f:
        for item in batch:
            f.write(item + '\n')


  0%|          | 0/1863 [00:00<?, ?it/s]

100%|██████████| 1745/1745 [02:32<00:00, 11.47it/s]


In [ ]:
import json
data = []
with open("batch_gpt_prompts.jsonl", "r") as f:
    for line in f:
        data.append(json.loads(line))

In [ ]:
client = OpenAI(api_key='')
batch_input_files = []

for i in tqdm(range(1, 73)):
    batch_input_file = client.files.create(
                    file=open(f'batches/batch_{i}.jsonl', "rb"),
                    purpose="batch"
                )
    batch_input_files.append(batch_input_file)

  0%|          | 0/72 [00:00<?, ?it/s]

 18%|█▊        | 13/72 [12:12<55:25, 56.37s/it] 


KeyboardInterrupt: 

In [ ]:
# import asyncio
# from openai import AsyncOpenAI
# from tqdm.asyncio import tqdm_asyncio

# client = AsyncOpenAI(api_key='')

# async def upload_file(i):
#     filename = f"batches/batch_{i}.jsonl"
#     # Open in async context
#     async with await asyncio.to_thread(open, filename, "rb") as f:
#         return await client.files.create(file=f, purpose="batch")

# async def main():
#     tasks = [upload_file(i) for i in range(1, 73)]
#     results = []
#     async for result in tqdm_asyncio.as_completed(tasks, total=len(tasks)):
#         results.append(await result)
#     return results

# batch_input_files = asyncio.run(main())

import asyncio
from pathlib import Path
from io import BytesIO
from tqdm.asyncio import tqdm_asyncio
from openai import AsyncOpenAI

client = AsyncOpenAI(api_key='')

async def upload_file(i, sem):
    filename = Path(f"batches/batch_{i}.jsonl")
    data = await asyncio.to_thread(filename.read_bytes)  # non-blocking read
    bio = BytesIO(data)
    bio.name = filename.name  # give it a filename for multipart/form-data
    async with sem:
        return await client.files.create(file=bio, purpose="batch")

async def main(concurrency=8, start=1, end=72):
    sem = asyncio.Semaphore(concurrency)
    tasks = [upload_file(i, sem) for i in range(start, end+1)]
    return await tqdm_asyncio.gather(*tasks)

# ← In Jupyter/IPython, run with top-level await:
batch_input_files = await main(concurrency=16, start=1, end=72)

100%|██████████| 72/72 [05:14<00:00,  4.36s/it]


In [12]:
batch_input_files[0].id

'file-GxTUzsQxKD51mVaMBnza5t'

In [17]:
batch_objects = {}

In [ ]:
client = OpenAI(api_key='')

In [24]:
for batch_input_file in batch_input_files:
    id = batch_input_file.id
    if id not in batch_objects:
        batch_input_file_id = batch_input_file.id
        batch_objects[id] = client.batches.create(
            input_file_id=batch_input_file_id,
            endpoint="/v1/chat/completions",
            completion_window="24h",
            metadata={
                "id": id
            }
        )

In [25]:
batch_objects

{'file-GxTUzsQxKD51mVaMBnza5t': Batch(id='batch_68a55b0fcbc08190914fd4d6d4d332b6', completion_window='24h', created_at=1755667215, endpoint='/v1/chat/completions', input_file_id='file-GxTUzsQxKD51mVaMBnza5t', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1755753615, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'id': 'file-GxTUzsQxKD51mVaMBnza5t'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0)),
 'file-KKhhsAjwnyT35v4qNMiz4a': Batch(id='batch_68a55c05abe481909db2b6eeb67f886c', completion_window='24h', created_at=1755667461, endpoint='/v1/chat/completions', input_file_id='file-KKhhsAjwnyT35v4qNMiz4a', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1755753861, failed_at=None, finalizing_at=None, in_progress_at=N

In [40]:
count_dict = defaultdict(int)

for key, value in batch_objects.items():
    count_dict[client.batches.retrieve(value.id).status] += 1
    # if client.batches.retrieve(value.id).status == "completed":
    #     print("key", key)
    # print(client.batches.retrieve(value.id))

print(count_dict)

defaultdict(<class 'int'>, {'completed': 72})


In [42]:
for key, _ in batch_objects.items():
        batch_object = batch_objects[key]
        output_file_id = client.batches.retrieve(batch_object.id).output_file_id
        output_response = client.files.content(output_file_id)
        os.makedirs("outfiles", exist_ok=True)
        with open(f"./outfiles/{output_file_id}.jsonl", 'w') as f:
                f.write(output_response.text)

In [43]:
file_ids = []

In [44]:
for key, _ in batch_objects.items():
        batch_object = batch_objects[key]
        output_file_id = client.batches.retrieve(batch_object.id).output_file_id
        file_ids.append(output_file_id)

In [58]:
mapping={"A":1, "B":2, "C":3, "D":4, "E":5}
import pandas as pd
actionswap_df = pd.read_csv("/n/fs/visualai-scr/temp_LLP/ellie/slowfast_kinetics/dataset/action_swap/action_swap_mcq_all_fields.csv")

In [102]:
result_dict = defaultdict(lambda:defaultdict(int))

In [96]:
path_list = []

In [103]:
for index, file_id in enumerate(file_ids):
    print("Starting index", index)
    path = f"outfiles/{file_id}.jsonl"
    data = []
    with open(path, "r") as f:
        for line in f:
            data.append(json.loads(line))
    
    for row in tqdm(data):
        custom_id = row["custom_id"]
        response = row["response"]["body"]["choices"][0]["message"]["content"][0]

        prompt_num = custom_id.split(":")[0]
        action_swap_path = custom_id.split(":")[1]

        if prompt_num == "2":
            path_list.append(action_swap_path)

        row = actionswap_df[actionswap_df["action_swap_path"] == action_swap_path]
        human_choice = row["human_choice"].item()
        background_choice = row["background_choice"].item()

        if response in mapping:
            if mapping[response] == human_choice:
                result_dict[prompt_num]["pred_human"] += 1
            elif mapping[response] == background_choice:
                result_dict[prompt_num]["pred_background"] += 1

        result_dict[prompt_num]["total"] += 1



Starting index 0


100%|██████████| 228/228 [00:00<00:00, 2472.01it/s]


Starting index 1


100%|██████████| 273/273 [00:00<00:00, 2472.11it/s]


Starting index 2


...

100%|██████████| 213/213 [00:00<00:00, 2457.01it/s]


Starting index 70


100%|██████████| 212/212 [00:00<00:00, 2476.15it/s]


Starting index 71


100%|██████████| 34/34 [00:00<00:00, 2317.67it/s]


In [101]:
print(len(path_list))

1745


In [104]:
for key, value in result_dict.items():
    print("Prompt", key, "Human-Bg", value["pred_human"], value["pred_background"], "Total:", value["total"])

Prompt 2 Human-Bg 790 760 Total: 1745
Prompt 4 Human-Bg 795 757 Total: 1745
Prompt 5 Human-Bg 804 756 Total: 1745
Prompt 6 Human-Bg 801 748 Total: 1743
Prompt 8 Human-Bg 802 725 Total: 1744
Prompt 11 Human-Bg 799 763 Total: 1744
Prompt 13 Human-Bg 780 703 Total: 1651
Prompt 15 Human-Bg 815 725 Total: 1745
Prompt 16 Human-Bg 811 746 Total: 1745
Prompt 18 Human-Bg 802 727 Total: 1744
Prompt 12 Human-Bg 24 45 Total: 81
